In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-CSE-CIC-IDS2018-v3.parquet"
data = pd.read_parquet(file_name)

In [3]:
##ADDED
file_name_test = "NF-CSE-CIC-IDS2018-v2.parquet"
test_data = pd.read_parquet(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    17514626
1     2600903
Name: count, dtype: int64

In [5]:
## ADDED
test_data.Label.value_counts()
##

Label
0    16635567
1     2258141
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
##ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
# ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['Benign', 'FTP-BruteForce', 'SSH-Bruteforce',
       'DoS_attacks-GoldenEye', 'DoS_attacks-Slowloris',
       'DoS_attacks-SlowHTTPTest', 'DoS_attacks-Hulk',
       'DDoS_attacks-LOIC-HTTP', 'DDOS_attack-LOIC-UDP',
       'DDOS_attack-HOIC', 'Brute_Force_-Web', 'Brute_Force_-XSS',
       'SQL_Injection', 'Infilteration', 'Bot'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['SSH-Bruteforce', 'Benign', 'DDoS attacks-LOIC-HTTP',
       'DDOS attack-HOIC', 'DoS attacks-Slowloris', 'DoS attacks-Hulk',
       'FTP-BruteForce', 'Infilteration', 'Bot', 'DoS attacks-GoldenEye',
       'Brute Force -Web', 'DoS attacks-SlowHTTPTest', 'SQL Injection',
       'DDOS attack-LOIC-UDP', 'Brute Force -XSS'], dtype=object)

In [12]:
## ADDED
test_data['Attack'] = test_data['Attack'].str.replace(' ', '_')
##

In [13]:
data = data.groupby(by='Attack').sample(frac=0.02, random_state=13)

In [14]:
## ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.02, random_state=13)
##

In [15]:
data.groupby(by="Attack").count()

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,350293,350293,350293,350293,350293,350293,350293,350293,350293,350293,...,350293,350293,350293,350293,350293,350293,350293,350293,350293,350293
Bot,4154,4154,4154,4154,4154,4154,4154,4154,4154,4154,...,4154,4154,4154,4154,4154,4154,4154,4154,4154,4154
Brute_Force_-Web,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
Brute_Force_-XSS,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
DDOS_attack-HOIC,20646,20646,20646,20646,20646,20646,20646,20646,20646,20646,...,20646,20646,20646,20646,20646,20646,20646,20646,20646,20646
DDOS_attack-LOIC-UDP,69,69,69,69,69,69,69,69,69,69,...,69,69,69,69,69,69,69,69,69,69
DDoS_attacks-LOIC-HTTP,5772,5772,5772,5772,5772,5772,5772,5772,5772,5772,...,5772,5772,5772,5772,5772,5772,5772,5772,5772,5772
DoS_attacks-GoldenEye,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226,...,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226
DoS_attacks-Hulk,2002,2002,2002,2002,2002,2002,2002,2002,2002,2002,...,2002,2002,2002,2002,2002,2002,2002,2002,2002,2002


In [16]:
##ADDED
test_data.groupby(by="Attack").count()
##

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711,...,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711
Bot,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862,...,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862
Brute_Force_-Web,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
Brute_Force_-XSS,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
DDOS_attack-HOIC,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617,...,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617
DDOS_attack-LOIC-UDP,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
DDoS_attacks-LOIC-HTTP,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146,...,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146
DoS_attacks-GoldenEye,554,554,554,554,554,554,554,554,554,554,...,554,554,554,554,554,554,554,554,554,554
DoS_attacks-Hulk,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653,...,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653


In [17]:
# X = data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
#                        "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
#                        "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
#                        "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
# y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
##Added
X = test_data.drop(columns=["Attack", "Label"])
y = test_data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)
##

In [18]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [19]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [20]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [21]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
9910620,172.31.65.112,172.217.15.65,6.210352e-09,1.019912e-10,0.000043,5.354863e-07,0.000117,4.094895e-07,1.624410e-08,1.157705e-08,...,6.299838e-08,0.000258,0.001348,3.704528e-09,3.704528e-09,6.190550e-09,6.190550e-09,0.000000,3.764741e-09,"[6.210351872620524e-09, 1.0199124955581988e-10..."
17557027,18.218.11.51,172.31.69.28,4.579454e-08,1.543672e-07,0.000122,1.161360e-06,0.000266,1.161360e-06,1.197824e-07,8.536806e-08,...,0.000000e+00,0.015222,0.006244,2.731684e-08,2.731684e-08,4.564852e-08,4.564852e-08,0.000000,2.776084e-08,"[4.57945440200515e-08, 1.5436721751920067e-07,..."
4328285,190.38.79.218,172.31.64.101,4.079400e-08,1.778170e-09,0.000103,1.034545e-06,0.000071,8.276362e-07,1.644264e-09,2.215507e-09,...,0.000000e+00,0.003398,0.001695,2.433397e-08,2.433397e-08,4.066392e-08,4.066392e-08,0.000000,2.472949e-08,"[4.0793998038229324e-08, 1.7781696021675934e-0..."
1287389,172.31.68.15,172.31.0.2,6.533832e-09,5.154645e-09,0.000038,5.997981e-07,0.000067,5.997981e-07,6.565366e-09,6.565366e-09,...,0.000000e+00,0.000000,0.000000,7.054050e-08,7.054050e-08,6.235997e-08,5.346222e-09,0.000036,7.168706e-08,"[6.533831560321106e-09, 5.154645331031907e-09,..."
4527816,172.31.66.21,172.31.0.2,1.168657e-08,9.219721e-09,0.000079,1.072813e-06,0.000097,1.072813e-06,1.174297e-08,1.174297e-08,...,0.000000e+00,0.000000,0.000000,1.261704e-07,1.261704e-07,1.115385e-07,9.562381e-09,0.000064,1.282212e-07,"[1.1686566408769841e-08, 9.219721141969458e-09..."


In [22]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [23]:
# Training graph
train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [24]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [25]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [26]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [27]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [28]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [29]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [30]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [31]:
# Convert to GPU
train_g = train_g.to('cuda')

In [32]:
dgi.load_state_dict(torch.load('best_dgi_CSE_v3.pkl'))

/tmp/ipykernel_59809/204607001.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_CSE_v3.pkl'))


<All keys matched successfully>

In [33]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [34]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [35]:
# Convert to GPU
test_g = test_g.to('cuda')

In [36]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [37]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [38]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,0.008314,0.031015,0.015412,0.003768,-0.002465,0.042314,0.160423,0.050804,0.013874,-0.059009,...,-0.018326,0.009919,0.098686,-0.058157,0.070793,-0.000830,-0.101493,-0.042032,Benign,0
1,0.013268,0.014777,0.037946,0.005275,0.013540,0.059844,0.164183,0.042322,0.026008,-0.066446,...,-0.023653,0.017025,0.110774,-0.033647,0.071297,-0.002327,-0.122153,-0.038432,Benign,0
2,0.013268,0.014777,0.037946,0.005275,0.013540,0.059844,0.164183,0.042322,0.026008,-0.066446,...,-0.023653,0.017025,0.110774,-0.033647,0.071297,-0.002327,-0.122153,-0.038432,Benign,0
3,0.013268,0.014777,0.037946,0.005275,0.013540,0.059844,0.164183,0.042322,0.026008,-0.066446,...,-0.023653,0.017025,0.110774,-0.033647,0.071297,-0.002327,-0.122153,-0.038432,Benign,0
4,0.013268,0.014777,0.037946,0.005275,0.013540,0.059844,0.164183,0.042322,0.026008,-0.066446,...,-0.023653,0.017025,0.110774,-0.033647,0.071297,-0.002327,-0.122153,-0.038432,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528327,0.012887,0.018900,0.035515,0.005757,0.010293,0.058653,0.169105,0.040529,0.026302,-0.066856,...,-0.020280,0.013748,0.110048,-0.034575,0.068771,-0.003084,-0.123840,-0.040623,Benign,0
528328,0.003625,0.008979,0.047020,0.000726,0.024550,0.050395,0.143478,0.055110,0.019236,-0.064298,...,-0.025270,0.001019,0.092490,-0.025227,0.061965,-0.000566,-0.111106,-0.037465,Benign,0
528329,0.023258,0.024358,0.073502,-0.039268,0.041752,0.087018,0.181286,0.074837,0.022228,-0.050223,...,-0.038504,0.002299,0.117326,-0.057304,0.086255,-0.042282,-0.152486,-0.049590,Benign,0
528330,0.009183,0.014755,0.040691,0.003463,0.014551,0.056293,0.158235,0.047672,0.022988,-0.066331,...,-0.024306,0.008959,0.104839,-0.030715,0.067149,-0.004063,-0.118188,-0.039897,Benign,0


# Embeddings CBLOF  Embeddings

In [39]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [40]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [41]:
test_samples

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.009930,0.017101,0.039014,0.004768,0.013314,0.056822,0.163110,0.044734,0.024188,-0.066541,...,-0.082234,0.010126,-0.021135,0.009293,0.105494,-0.031630,0.066452,-0.003356,-0.120788,-0.040572
1,0.011258,0.017385,0.037787,0.004620,0.011181,0.058316,0.164667,0.044453,0.024869,-0.066719,...,-0.083372,0.011641,-0.022794,0.012506,0.109239,-0.033378,0.068181,-0.004250,-0.121786,-0.040812
2,0.011404,0.017253,0.037712,0.004528,0.010853,0.058597,0.164854,0.044397,0.024890,-0.066672,...,-0.083790,0.011804,-0.023154,0.012798,0.109743,-0.033710,0.068577,-0.004582,-0.121913,-0.040591
3,0.012313,0.013901,0.039853,0.004684,0.015494,0.058560,0.159363,0.044272,0.026147,-0.065425,...,-0.079186,0.008207,-0.021390,0.013790,0.106188,-0.029624,0.066409,-0.001660,-0.123067,-0.038366
4,0.012669,0.013680,0.038913,0.004951,0.014926,0.059097,0.160247,0.044076,0.025264,-0.065653,...,-0.082317,0.010357,-0.023457,0.016004,0.108192,-0.032546,0.070653,-0.001943,-0.120428,-0.037622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226421,0.004128,0.008579,0.047206,0.000757,0.024353,0.051281,0.144055,0.054768,0.019611,-0.064572,...,-0.087156,-0.003010,-0.026078,0.002247,0.093897,-0.025434,0.062876,-0.000872,-0.111575,-0.037286
226422,0.011397,0.016587,0.038840,0.004855,0.012255,0.059037,0.165053,0.043808,0.025260,-0.067133,...,-0.082741,0.011620,-0.022977,0.012746,0.109624,-0.032401,0.068783,-0.004126,-0.122218,-0.040485
226423,0.022104,0.022737,0.076637,-0.039106,0.045093,0.087124,0.179698,0.074484,0.022686,-0.050916,...,-0.125601,-0.025975,-0.036950,-0.000067,0.114598,-0.053083,0.084114,-0.040851,-0.153419,-0.048956
226424,0.005355,0.010440,0.044866,0.001800,0.021023,0.052295,0.147894,0.052830,0.020343,-0.065177,...,-0.086972,0.000269,-0.025810,0.004160,0.097024,-0.027228,0.064263,-0.001691,-0.112576,-0.038089


In [42]:
df_raw_train = pd.concat([X_train.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_train], axis=1)
df_raw_test = pd.concat([X_test.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_test], axis=1)

In [43]:
df_raw_train

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack,Label
9910620,6.210352e-09,1.019912e-10,0.000043,5.354863e-07,0.000117,4.094895e-07,1.624410e-08,1.157705e-08,1.508822e-08,0.000000,...,0.000258,0.001348,3.704528e-09,3.704528e-09,6.190550e-09,6.190550e-09,0.000000,3.764741e-09,Benign,0
17557027,4.579454e-08,1.543672e-07,0.000122,1.161360e-06,0.000266,1.161360e-06,1.197824e-07,8.536806e-08,1.112591e-07,0.997593,...,0.015222,0.006244,2.731684e-08,2.731684e-08,4.564852e-08,4.564852e-08,0.000000,2.776084e-08,DDOS_attack-HOIC,1
4328285,4.079400e-08,1.778170e-09,0.000103,1.034545e-06,0.000071,8.276362e-07,1.644264e-09,2.215507e-09,1.297968e-10,0.000000,...,0.003398,0.001695,2.433397e-08,2.433397e-08,4.066392e-08,4.066392e-08,0.000000,2.472949e-08,Benign,0
1287389,6.533832e-09,5.154645e-09,0.000038,5.997981e-07,0.000067,5.997981e-07,6.565366e-09,6.565366e-09,6.697107e-09,0.000000,...,0.000000,0.000000,7.054050e-08,7.054050e-08,6.235997e-08,5.346222e-09,0.000036,7.168706e-08,Benign,0
4527816,1.168657e-08,9.219721e-09,0.000079,1.072813e-06,0.000097,1.072813e-06,1.174297e-08,1.174297e-08,1.197860e-08,0.000000,...,0.000000,0.000000,1.261704e-07,1.261704e-07,1.115385e-07,9.562381e-09,0.000064,1.282212e-07,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7498611,5.044341e-09,4.236683e-09,0.000032,4.630646e-07,0.000072,4.630646e-07,5.068686e-09,5.068686e-09,5.170395e-09,0.000000,...,0.000000,0.000000,5.445967e-08,5.445967e-08,4.679600e-08,4.127466e-09,0.000028,5.534486e-08,Benign,0
1455276,1.961397e-08,6.611604e-08,0.000023,4.974148e-07,0.000113,3.979319e-07,1.556355e-08,1.476538e-08,4.765268e-08,0.427273,...,0.000815,0.002674,1.169990e-08,1.169990e-08,1.955143e-08,1.955143e-08,0.000000,1.189007e-08,DDoS_attacks-LOIC-HTTP,1
10551368,4.350282e-09,7.144373e-11,0.000049,3.751021e-07,0.000115,3.751021e-07,6.228879e-11,3.274889e-09,1.003687e-10,0.000000,...,0.000181,0.000944,2.594980e-09,2.594980e-09,4.336411e-09,4.336411e-09,0.000000,2.637159e-09,Benign,0
1020180,1.263220e-07,4.258143e-07,0.000110,2.562843e-06,0.000059,1.281422e-06,6.551236e-10,8.914238e-09,5.746286e-10,0.000000,...,0.005249,0.018709,7.535213e-08,7.535213e-08,1.259192e-07,1.259192e-07,0.000000,7.657690e-08,Benign,0


In [44]:
raw_benign_train_samples = df_raw_train[df_raw_train.Label == 0].drop(columns=["Label", "Attack"])
raw_normal_train_samples = df_raw_train.drop(columns=["Label", "Attack"])

raw_train_labels = df_raw_train["Label"]
raw_test_labels = df_raw_test["Label"]

raw_test_samples = df_raw_test.drop(columns=["Label", "Attack"])

In [45]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [46]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    try:
        clf_if.fit(benign_train_samples)
    except Exception as e:
        continue
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [03:24<00:00,  5.67s/it]

{'n_estimators': 10, 'con': 0.001}
0.9573322971296101
              precision    recall  f1-score   support

           0     0.9830    0.9978    0.9904    199328
           1     0.9821    0.8729    0.9243     27098

    accuracy                         0.9829    226426
   macro avg     0.9826    0.9354    0.9573    226426
weighted avg     0.9829    0.9829    0.9825    226426



In [47]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    try:
        clf_if.fit(normal_train_samples)
    except Exception as e:
        continue
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [03:32<00:00,  5.92s/it]

{'n_estimators': 2, 'con': 0.1}
0.9443309476442819
              precision    recall  f1-score   support

           0     0.9775    0.9982    0.9877    199328
           1     0.9841    0.8308    0.9009     27098

    accuracy                         0.9781    226426
   macro avg     0.9808    0.9145    0.9443    226426
weighted avg     0.9783    0.9781    0.9773    226426



In [48]:
###  CBLOF RAW

In [49]:
from pyod.models.cblof import CBLOF

n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = CBLOF(n_clusters=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

  8%|▊         | 3/36 [00:02<00:21,  1.56it/s]

2


 11%|█         | 4/36 [00:02<00:15,  2.06it/s]

2


...

3


 33%|███▎      | 12/36 [00:06<00:09,  2.47it/s]

3


100%|██████████| 36/36 [00:24<00:00,  1.47it/s]

{'n_estimators': 10, 'con': 0.04}
0.8738090690863115
              precision    recall  f1-score   support

           0     0.9761    0.9600    0.9680     99814
           1     0.7375    0.8269    0.7796     13549

    accuracy                         0.9441    113363
   macro avg     0.8568    0.8934    0.8738    113363
weighted avg     0.9476    0.9441    0.9455    113363



In [50]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

 25%|██▌       | 9/36 [00:06<00:16,  1.68it/s]

3


 28%|██▊       | 10/36 [00:06<00:13,  1.96it/s]

3


 31%|███       | 11/36 [00:06<00:11,  2.25it/s]

3


 33%|███▎      | 12/36 [00:07<00:09,  2.51it/s]

3


100%|██████████| 36/36 [00:26<00:00,  1.35it/s]

benign only
{'n_estimators': 2}
0.8611859140799868
              precision    recall  f1-score   support

           0     0.9590    0.9802    0.9695     99814
           1     0.8261    0.6916    0.7529     13549

    accuracy                         0.9457    113363
   macro avg     0.8925    0.8359    0.8612    113363
weighted avg     0.9431    0.9457    0.9436    113363



In [51]:
# HBOS  Embeddings

In [52]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [53]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [04:43<00:00,  7.88s/it]

{'n_estimators': 10, 'con': 0.001}
0.9605767255895168
              precision    recall  f1-score   support

           0     0.9830    0.9994    0.9911    199328
           1     0.9952    0.8728    0.9300     27098

    accuracy                         0.9843    226426
   macro avg     0.9891    0.9361    0.9606    226426
weighted avg     0.9845    0.9843    0.9838    226426



In [54]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [05:09<00:00,  8.61s/it]

{'n_estimators': 5, 'con': 0.1}
0.8244234197140611
              precision    recall  f1-score   support

           0     0.9470    0.9805    0.9635    199328
           1     0.8062    0.5961    0.6854     27098

    accuracy                         0.9345    226426
   macro avg     0.8766    0.7883    0.8244    226426
weighted avg     0.9301    0.9345    0.9302    226426



In [55]:
##  HBOS  RAw

In [56]:
from pyod.models.cblof import CBLOF

n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = HBOS(n_bins=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:21<00:00,  1.71it/s]

{'n_estimators': 30, 'con': 0.04}
0.8268302258771507
              precision    recall  f1-score   support

           0     0.9583    0.9591    0.9587     99814
           1     0.6971    0.6927    0.6949     13549

    accuracy                         0.9273    113363
   macro avg     0.8277    0.8259    0.8268    113363
weighted avg     0.9271    0.9273    0.9272    113363



In [57]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = HBOS(n_bins=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:21<00:00,  1.68it/s]

benign only
{'n_estimators': 5}
0.7805213661279453
              precision    recall  f1-score   support

           0     0.9764    0.8878    0.9300     99814
           1     0.5046    0.8420    0.6310     13549

    accuracy                         0.8823    113363
   macro avg     0.7405    0.8649    0.7805    113363
weighted avg     0.9200    0.8823    0.8943    113363



In [58]:
##  PCA  Emb

In [59]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [01:47<00:00,  2.99s/it]

{'n_estimators': 10, 'con': 0.001}
0.9575351981060627
              precision    recall  f1-score   support

           0     0.9830    0.9980    0.9904    199328
           1     0.9830    0.8728    0.9247     27098

    accuracy                         0.9830    226426
   macro avg     0.9830    0.9354    0.9575    226426
weighted avg     0.9830    0.9830    0.9825    226426



In [60]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:46<00:00,  2.97s/it]

{'n_estimators': 15, 'con': 0.1}
0.9269372899233412
              precision    recall  f1-score   support

           0     0.9744    0.9933    0.9838    199328
           1     0.9425    0.8080    0.8701     27098

    accuracy                         0.9711    226426
   macro avg     0.9585    0.9007    0.9269    226426
weighted avg     0.9706    0.9711    0.9702    226426



In [61]:
##  PCA  RAw

In [62]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples)
   
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:14<00:00,  2.49it/s]

{'n_estimators': 30, 'con': 0.1}
0.7675541585434458
              precision    recall  f1-score   support

           0     0.9636    0.9006    0.9311     99814
           1     0.5059    0.7495    0.6041     13549

    accuracy                         0.8826    113363
   macro avg     0.7347    0.8251    0.7676    113363
weighted avg     0.9089    0.8826    0.8920    113363



In [63]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples)

    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:15<00:00,  2.38it/s]

benign only
{'n_estimators': 5}
0.7265300185324359
              precision    recall  f1-score   support

           0     0.9587    0.8705    0.9125     99814
           1     0.4314    0.7239    0.5406     13549

    accuracy                         0.8530    113363
   macro avg     0.6950    0.7972    0.7265    113363
weighted avg     0.8957    0.8530    0.8680    113363



In [64]:
##  IF  Emb

In [65]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [01:17<00:00,  3.22s/it]

{'n_estimators': 150, 'con': 0.001}
0.9444252903645183
              precision    recall  f1-score   support

           0     0.9772    0.9986    0.9878    199328
           1     0.9874    0.8287    0.9011     27098

    accuracy                         0.9782    226426
   macro avg     0.9823    0.9136    0.9444    226426
weighted avg     0.9784    0.9782    0.9774    226426



In [66]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [01:07<00:00,  2.81s/it]

{'n_estimators': 20, 'con': 0.1}
0.8725350876560137
              precision    recall  f1-score   support

           0     0.9746    0.9615    0.9680    199328
           1     0.7422    0.8154    0.7771     27098

    accuracy                         0.9440    226426
   macro avg     0.8584    0.8885    0.8725    226426
weighted avg     0.9468    0.9440    0.9451    226426



In [67]:
##  IF  Raw

In [68]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples.to_numpy())
   
    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:22<00:00,  1.06it/s]

{'n_estimators': 50, 'con': 0.04}
0.8392491044985002
              precision    recall  f1-score   support

           0     0.9629    0.9591    0.9610     99814
           1     0.7074    0.7278    0.7175     13549

    accuracy                         0.9315    113363
   macro avg     0.8352    0.8435    0.8392    113363
weighted avg     0.9324    0.9315    0.9319    113363



In [69]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples.to_numpy())

    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:24<00:00,  1.01s/it]

benign only
{'n_estimators': 150}
0.7406585779159964
              precision    recall  f1-score   support

           0     0.9631    0.8755    0.9172     99814
           1     0.4509    0.7531    0.5641     13549

    accuracy                         0.8609    113363
   macro avg     0.7070    0.8143    0.7407    113363
weighted avg     0.9019    0.8609    0.8750    113363

